# Create Potential Field

## Load b_bottom

In [ ]:
import os

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= "3"

In [ ]:
import json

In [ ]:
with open('config.json') as config:
    info = json.load(config)

nx = info['nx']
ny = info['ny']
nz = info['nz']
b_norm = info['b_norm']
spatial_norm = info['spatial_norm']

input_path = info['input_path']

In [ ]:
import pickle

In [ ]:
input_original = os.path.join(input_path, 'original')
b_bottom_original_path = os.path.join(input_original, "b_bottom_original.pickle")

with open(b_bottom_original_path, "rb") as f:
    b_bottom = pickle.load(f)

## Create Potential Field

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from tqdm import tqdm

In [ ]:
from zpinn.pinn_nf2_cleanup import PotentialModel, create_coordinates

In [ ]:
bottom_values = b_bottom.reshape(-1, 3)
bottom_bounds = (0, nx-1, 0, ny-1, 0, 0)
bottom_coords = create_coordinates(bottom_bounds).reshape(-1, 3)

In [ ]:
top_bounds = (0, nx-1, 0, ny-1, nz-1, nz-1)
lateral_bounds_1 = (0, 0, 0, ny-1, 0, nz-1)
lateral_bounds_2 = (nx-1, nx-1, 0, ny-1, 0, nz-1)
lateral_bounds_3 = (0, nx-1, 0, 0, 0, nz-1)
lateral_bounds_4 = (0, nx-1, ny-1, ny-1, 0, nz-1)

top_coords = create_coordinates(top_bounds)
lateral_coords_1 = create_coordinates(lateral_bounds_1)
lateral_coords_2 = create_coordinates(lateral_bounds_2)
lateral_coords_3 = create_coordinates(lateral_bounds_3)
lateral_coords_4 = create_coordinates(lateral_bounds_4)

In [ ]:
top_lateral_coordinates = [top_coords,
                           lateral_coords_1,
                           lateral_coords_2,
                           lateral_coords_3,
                           lateral_coords_4]

In [ ]:
def get_potential_boundary(b_n, height, batch_size=2048, **kwargs):
    assert not np.any(np.isnan(b_n)), 'Invalid data value'

    cube_shape = (*b_n.shape, height)

    b_n = b_n.reshape((-1)).astype(np.float32)
    coords = [np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], cube_shape[2] - 2:cube_shape[2] + 1], -1),       #top
              np.stack(np.mgrid[:cube_shape[0], -1:2, :cube_shape[2]], -1),                                      #y=0
              np.stack(np.mgrid[:cube_shape[0], cube_shape[1] - 2:cube_shape[1] + 1, :cube_shape[2]], -1),       #y=max
              np.stack(np.mgrid[-1:2, :cube_shape[1], :cube_shape[2]], -1),                                      #x=0
              np.stack(np.mgrid[cube_shape[0] - 2:cube_shape[0] + 1, :cube_shape[1], :cube_shape[2]], -1), ]     #x=max
    fields = _compute_fields(coords, cube_shape, b_n, batch_size=batch_size, **kwargs)

    fields = [fields[0][:, :, 1],
              fields[1][:, 1, :], fields[2][:, 1, :],
              fields[3][1, :, :], fields[4][1, :, :]]
    coords = [coords[0][:, :, 1],
              coords[1][:, 1, :], coords[2][:, 1, :],
              coords[3][1, :, :], coords[4][1, :, :]]
    return coords, fields

def _compute_fields(coords, cube_shape, b_n, batch_size=2048, progress=False):
    coords_shape = [c.shape[:-1] for c in coords]
    flat_coords = np.concatenate([c.reshape(((-1, 3))) for c in coords])

    r_p = np.stack(np.mgrid[:cube_shape[0], :cube_shape[1], :1], -1).reshape((-1, 3))

    # torch code
    # r = (x * y, 3); coords = (x*y*z, 3), c = (1, 3)
    # --> (x * y, x * y * z, 3) --> (x * y, x * y * z) --> (x * y * z)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    with torch.no_grad():
        b_n = torch.tensor(b_n, dtype=torch.float32, )
        r_p = torch.tensor(r_p, dtype=torch.float32, )
        model = nn.DataParallel(PotentialModel(b_n, r_p, )).to(device)

        flat_coords = torch.tensor(flat_coords, dtype=torch.float32, )

        potential = []
        iter = DataLoader(TensorDataset(flat_coords), batch_size=batch_size, num_workers=2)
        iter = iter if progress else tqdm(iter, desc='Potential Field')
        for coord, in iter:
            coord = coord.to(device)
            p_batch = model(coord)
            potential += [p_batch.cpu()]

    potential = torch.cat(potential).numpy()
    idx = 0
    fields = []
    for s in coords_shape:
        p = potential[idx:idx + np.prod(s)].reshape(s)
        b = - 1 * np.stack(np.gradient(p, axis=[0, 1, 2], edge_order=2), axis=-1)
        fields += [b]
        idx += np.prod(s)

    return fields

In [ ]:
coords, fields = get_potential_boundary(b_bottom[:, :, 2], nz)

Potential Field: 100%|██████████| 480/480 [00:20<00:00, 23.94it/s]


In [ ]:
for i in range(len(coords)):
    coords[i] = np.reshape(coords[i], (-1, 3))
    fields[i] = np.reshape(fields[i], (-1, 3))

In [ ]:
top_lateral_values = np.concatenate(fields) 
top_lateral_coords = np.concatenate(coords)

boundary_values = np.concatenate([top_lateral_values, bottom_values])
boundary_coords = np.concatenate([top_lateral_coords, bottom_coords])

normalized_boundary_values = boundary_values / b_norm
normalized_boundary_coords = boundary_coords / spatial_norm

boundary_data = np.stack([normalized_boundary_coords, normalized_boundary_values], 1)

In [ ]:
bp_top = fields[0]
bp_lateral_3 = fields[1]
bp_lateral_4 = fields[2]
bp_lateral_1 = fields[3]
bp_lateral_2 = fields[4]

In [ ]:
b_bottom = b_bottom / b_norm
bp_top = bp_top / b_norm
bp_lateral_1 = bp_lateral_1 / b_norm
bp_lateral_2 = bp_lateral_2 / b_norm
bp_lateral_3 = bp_lateral_3 / b_norm 
bp_lateral_4 = bp_lateral_4 / b_norm

In [ ]:
b_bottom_path = os.path.join(input_path, "b_bottom.pickle")
bp_top_path = os.path.join(input_path, "bp_top.pickle")
bp_lateral_1_path = os.path.join(input_path, "bp_lateral_1.pickle")
bp_lateral_2_path = os.path.join(input_path, "bp_lateral_2.pickle")
bp_lateral_3_path = os.path.join(input_path, "bp_lateral_3.pickle")
bp_lateral_4_path = os.path.join(input_path, "bp_lateral_4.pickle")
boundary_data_path = os.path.join(input_path, "boundary_data.pickle")

In [ ]:
with open(b_bottom_path,"wb") as f:
    pickle.dump(b_bottom, f)

with open(bp_top_path,"wb") as f:
    pickle.dump(bp_top, f)

with open(bp_lateral_1_path,"wb") as f:
    pickle.dump(bp_lateral_1, f)

with open(bp_lateral_2_path,"wb") as f:
    pickle.dump(bp_lateral_2, f)
    
with open(bp_lateral_3_path,"wb") as f:
    pickle.dump(bp_lateral_3, f)

with open(bp_lateral_4_path,"wb") as f:
    pickle.dump(bp_lateral_4, f)

with open(boundary_data_path,"wb") as f:
    pickle.dump(boundary_data, f)